# Routing with FastAPI

## Introduction

Conventionally, when working on a project, the code is partitioned into smaller scripts to ensure organisation. In FastAPI, this can be achieved using Routers. As an example, consider a case where you add two GET responses to your API: one for a regular DOB request and another for supplying an icon to the tab on which you are working:

In [ ]:
%%python
import fastapi
import uvicorn

api = fastapi.FastAPI()

@api.get('/')
def index():
    return 'Welcome to the celebrity DOB API!'

@api.get('/api/dob')
def dob():
    return 'Some random DOB'
    
if __name__ == '__main__':
    uvicorn.run(api, port=8008, host='127.0.0.1')

To view the output, visit `http://127.0.0.1:8008/api/dob`.
<p align=center><img src=images/Routing_1.png width=300></p>

One approach for separating these functions would be having one script for the home page and another for additional pages (`/api/dob`):

```
root/
│
├── home.py
├── api
│   └── dob_api.py
└── main.py
```

In [ ]:
# views/home.py

import fastapi

router = fastapi.APIRouter()


@router.get('/')
def index():
    return 'Welcome to the celebrity DOB API!'


In [ ]:
# api/dob_api.py

import fastapi

router = fastapi.APIRouter()

@router.get('/api/dob')
def dob():
    return 'Some random DOB'

In [ ]:
# main.py
import fastapi
import uvicorn
import home

from api import dob_api

api = fastapi.FastAPI()


def configure_routing():
    api.include_router(home.router)
    api.include_router(dob_api.router)


if __name__ == '__main__':
    configure_routing()
    uvicorn.run(api, port=8000, host='127.0.0.1')


Observe that in `home.py` and `api/dob_api.py`, we included `router = fastapi.APIRouter()`. As a result, all the files have a common node to search for all requests.

## Introducing the Pydantic Model

APIs accept request parameters, and the Pydantic model is responsible for casting these parameters to the correct type. In the above exercise, your built API should accept the first name and last name as parameters. 

Pydantic allows you to create a class where each attribute can be cast to a specified data type.

In [ ]:
from pydantic import BaseModel
from typing import Optional

class Celebrity(BaseModel):
    '''
    This class gives some information about a celebrity. It is intended to be 
    used with the FastAPI example.

    Attributes
    ----------
    first_name: str
        The first name of the celebrity
    last_name: str
        The last name of the celebrity
    city: bool
        If True, the API will also return the city where the celebrity was born.
    '''
    first_name: str
    last_name: str
    city: bool = False

uma_thurman = Celebrity(first_name='Uma', last_name='Thurman')    

Now, we create another directory to store our models:

```
root/
│
├── home.py
├── api
│   └── dob_api.py
├── models
│   └── celebrities.py
└── main.py
```

Thereafter, this Pydantic model can be employed in our FastAPI, as shown below:

In [ ]:
# models/celebrities.py

from pydantic import BaseModel
from typing import Optional

class Celebrity(BaseModel):
    '''
    This class gives some information about a celebrity. It is intended to be 
    used with the FastAPI example.

    Attributes
    ----------
    first_name: str
        The first name of the celebrity
    last_name: str
        The last name of the celebrity
    city: bool
        If True, the API will also return the city where the celebrity was born.
    '''
    first_name: str
    last_name: str
    city: bool = False

In [ ]:
# api/dob_api.py

import fastapi
from fastapi import Depends

from models.celebrities import Celebrity

router = fastapi.APIRouter()


@router.get('/api/dob/{first_name}')
def dob(celebrity: Celebrity = Depends()):
    full_name = f'{celebrity.first_name} {celebrity.last_name}'
    if celebrity.city:
        report = f'{full_name} was born a random day in a random city'
    else:
        report = f'{full_name} was born a random day'
    return report



Observe the `Depends` class in `celebrity`, used as an argument in `dob`. This tells the dob function that the variables it contains depend on an external class; therefore, it becomes aware that the attributes of the class are to be used as the query parameters.

With this, the following URL [http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman](http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman) will return the following:

<p align=center><img src=images/Routing_2.png width=400></p>

Currently, the DOB API is not meeting expectations. Thus, we create the real function that will scrape the data on the celebrity and return the date through our API. The function containing this service will be in the `services` directory:

```
root/
│
├── home.py
├── api
│   └── dob_api.py
├── models
│   └── celebrities.py
├── services
│   └── dob_service.py
└── main.py
```

In [ ]:
# services/dob_service.py
import requests
from bs4 import BeautifulSoup
import json
import re

def get_dob(first_name: str, last_name: str, city: bool=False):
    infobox_data = get_infobox(first_name, last_name)
    if not infobox_data:
        return None
    birthday = infobox_data.find('span', {'class': 'bday'})

    report = {'first name': first_name,
              'last_name': last_name,
              'Date of Birth': birthday.text}
    if city:
        birthplace = infobox_data.find('div', {'class': 'birthplace'})
        report['City'] = birthplace.text

    return report

def get_infobox(first_name: str, last_name: str):
    r = requests.get(f'https://en.wikipedia.org/wiki/{first_name}_{last_name}')
    soup = BeautifulSoup(r.text, 'html.parser')
    if soup.find_all('b', text=re.compile('Wikipedia does not have an article with this exact name')):
        return None
    celeb_infobox = soup.find('table', {'class': 'infobox biography vcard'})
    return celeb_infobox.find('td', {'class': 'infobox-data'})

In [ ]:
# api/dob_api.py

import fastapi
from fastapi import Depends
import json
from models.celebrities import Celebrity
# from services.dob_service import get_dob
from services.dob_service_async import get_dob
router = fastapi.APIRouter()

@router.get('/api/dob/{first_name}')
def dob(celebrity: Celebrity = Depends()):
    report = get_dob(celebrity.first_name, celebrity.last_name, celebrity.city)
    return fastapi.Response(content=json.dumps(report),
                            media_type='application/json')

If you visit the following URL [http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman&city=True](http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman&city=True), you will obtain a real API response.
<p align=center><img src=images/Routing_3.png width=500></p>

Further, the requests library can be employed to retrieve the uploaded JSON object and guarantee that everything works properly.

In [ ]:
requests.get('http://127.0.0.1:8008/api/dob/Uma?last_name=Thurman&city=True').json()

{'First name': 'Uma',
 'Last_name': 'Thurman',
 'Date of Birth': '1970-04-29',
 'City': 'Boston, Massachusetts, U.S.'}

## Conclusion
At this point, you should have a good understanding of
- how to route API requests to a single node called in main. 
- how to use Pydantic Model in FastAPI. 
    